# Ada Final Project - EDA

In [ ]:
# Python Standard Libraries
import re
import csv
import bz2
import json
import string

# Install using conda
# conda install matplotlib pandas ipywidgets beautifulsoup4 nltk
import nltk
#import torch
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import pandas as pd

import numpy as np
import plotly.express as px

from nltk.sentiment import SentimentIntensityAnalyzer
from bs4 import BeautifulSoup


# Not available through Conda
#!pip  install syllables transformers

import syllables

# BERT related modules
from transformers import BertTokenizer, BertModel


%matplotlib inline

nltk.download([
     "names",
     "stopwords",
     "averaged_perceptron_tagger",
     "vader_lexicon",
     "punkt",
 ]);

In [ ]:
PATH_TO_FILE = 'data/hillary-and-trump-quotes-2016.json.bz2'

CEFR_HTML_IN = "data/cefr_data.html"

CEFR_CLEAN_CSV_IN = "data/cefr_data_clean.csv"
CEFR_CSV_OUT = "data/cefr_data.csv"

CHUNK_SIZE = 100_000

RANDOM_SAMPLE_SIZE = 30_000

SEED = 92813

## 1. Load Data

### Initial Data Extraction

For reference we include the code we executed on Google Colab to extract all quotes by **Hillary Clinton** and **Donald Trump** during the **year 2016** from the Quotebank dataset. This was a one time operation, which is why we did it outside of this notebook. All other algorithms we apply to the data will be/have been possibly iterated on for improvement, which is where the notebook format comes in handy.

```python
PATH_TO_FILE = '/content/drive/MyDrive/Quotebank/quotes-2016.json.bz2'
PATH_TO_OUT = '/content/drive/MyDrive/hillary-and-trump-quotes-2016.json.bz2'

SPEAKER_NAMES = ['Hillary Clinton', 'Donald Trump']

hits = 0

with bz2.open(PATH_TO_FILE, 'rb') as s_file:
    with bz2.open(PATH_TO_OUT, 'wb') as d_file:

        for instance in s_file:

            instance = json.loads(instance)
            speaker = instance['speaker']

            if any(map(speaker.__contains__, desired_speakers)):

                d_file.write((json.dumps(instance)+'\n').encode('utf-8'))

                hits += 1


print(f"Hits: {hits}")
```

In [ ]:
df_1 = pd.read_json(PATH_TO_FILE, lines=True, compression='bz2') #chunksize=CHUNK_SIZE)

## 2. Enhance Data

In [ ]:
df_2 = df_1

df_2['proba'] = df_2['probas'].apply(lambda probas : float(probas[0][1]))

df_2.head()

## Samples

In [ ]:
sample = df_2.sample(n=RANDOM_SAMPLE_SIZE, random_state=SEED)

dt_sample = df_2[df_2['speaker'] == 'Donald Trump'].sample(n=RANDOM_SAMPLE_SIZE, random_state=SEED)
hc_sample = df_2[df_2['speaker'] == 'Hillary Clinton'].sample(n=RANDOM_SAMPLE_SIZE, random_state=SEED)

## 3. Clean Data

In [ ]:
df_3 = df_2

We are only interested in quotes by **Hillary Clinton** and **Donald Trump** during the **year 2016**, specifically from 01/01/2016 - 01/01/2017. The subset loaded only contains the Quotebank quotes which have one (or both) of them as a possible speaker in the _speaker_ columns list and that lie in the specified time frame.

Since the dataset was obtained using a ML model to extract and assign the quotes there will most likely be quotes which are faulty and quotes which have been assigned to the wrong speaker. The goal of the data cleaning is to remove such data points so that we can focus on working with as good data as possible.

We must specify what makes a quote faulty and motivate this so that we remove as many bad quotes as possible while not removing any or as little actually correct quotes as possible.

### 3.1 Filter: Remove quotes with low probas

**Motivation**

Certain quotes that the model assigned to Trump and Clinton have very low probabilities to actually be quotes by them inside of the text as by the computation of the model. We want to learn about the distribution of the probability of the assigned quotes so that we can take a decision on if and when to filter out certain quotes due to a too low probability computed for them by the model.

**Distribution of the Proba**

Here we plot the distribution of a sample of 1,000 Trump and 1,000 Clinton quotes respectively. The reason we sample seperatley is given the fact that we want to ensure a large enough sample pool for both candidates, which is needed since there are more quotes assigned to Trump than Clinton. We also plot the two seperatly to make sure that we do not miss any differences in the dsitribution.

In [ ]:
n_bins = 40
proba_bins = [round((1 / n_bins), 2) * i for i in range(0, n_bins  + 1)]

fig, axs = plt.subplots(2, 2, figsize=(10, 5))
fig.suptitle("Distribution of the probability computed by the model to the assigned speaker", fontsize=14)

axs[0, 0].set(xlim=(0, 1))
axs[0, 1].set(xlim=(0, 1))

axs[0, 0].hist(hc_sample['proba'], bins=proba_bins, color='b', alpha=0.4)
axs[0, 1].hist(dt_sample['proba'], bins=proba_bins, color='r', alpha=0.4)

axs[1, 0].set(xlim=(0, 1))
axs[1, 1].set(xlim=(0, 1))

axs[1, 0].boxplot(hc_sample['proba'], vert=False)
axs[1, 1].boxplot(dt_sample['proba'], vert=False)


axs[0, 0].set_ylabel('Frequency')
axs[0, 0].title.set_text('Hillary Clinton')

axs[0, 1].title.set_text('Donald Trump')

axs[1, 1].set_xlabel('Computed Probability')
axs[1, 0].set_xlabel('Computed Probability')

plt.show()
plt.clf()

print("Hillary Clinton Summary Statistics")
display(hc_sample['proba'].describe())
print()
print("Donald Trump Summary Statistics")
display(hc_sample['proba'].describe())

From plotting the distribution of probabilities which the model computed to the quotes it assigned to Trump and Hillary respectively we could now act in at least 3 ways:

1. Remove any quote which is below the min probability minus a small margin because they are outliers. **I.e. Only Remove outliers.**

2. Set the cutoff even higher because we decide to consider quotes with, ex. less than 0.4 probability assigned to the candidate being the speaker too weak to consider it in further analysis.

3. A further possibility could be to remove quotes, where the next best speaker assigned has a similair/close probability compare to the number one.
   
   Example: `[ [ 'Trump', 0.41 ], [ 'Kanye West', 0.39 ], ...]`

**TODO for later:** What should we do? What is scientifically sound? How do we motivate it?



**Filtering out datapoints with too low probability**

In [ ]:
PROBAS_THRESHOLD = 0.5

In [ ]:
df_3_1 = df_3

df_3_1 = df_3_1[df_3_1['proba'] >= PROBAS_THRESHOLD]

n_removed_lines = len(df_3) - len(df_3_1)
percentage_removed = (n_removed_lines / len(df_1)) * 100

print("Removed {:,.0f} datapoints or {:,.2f}% of the original dataset".format(n_removed_lines, percentage_removed))

### 3.2 Remove quotes by different Speakers

In [ ]:
df_3_1.groupby(['speaker']).count().sort_values('quotation', ascending=False)['quotation']

In [ ]:
df_3_2 = df_3_1

df_3_2 = df_3_2[df_3_2['speaker'].isin(['Hillary Clinton', 'Donald Trump'])]

display(df_3_2.groupby(['speaker']).count().sort_values('quotation', ascending=False)['quotation'])


n_removed_lines = len(df_3_1) - len(df_3_2)
percentage_removed = (n_removed_lines / len(df_1)) * 100

print("\n")

print("Removed {:,.0f} datapoints or {:,.2f}% of the original dataset".format(n_removed_lines, percentage_removed))

### 3.3 Filter: Remove Quotes with "nonsense" content

**TODO**: We should ideally check for the quality of the quotes. There are certainly some faulty quotes and maybe even gibberish in the dataset but it is hopefully very limited in scope. We should nonetheless attempt to look for faulty/gibbersih quotes which were extracted by the model and remove them.

## 4. Data Analysis

### New sampling with the cleaned data

In [ ]:
cleaned_sample = df_3_2.sample(n=RANDOM_SAMPLE_SIZE, random_state=SEED)

dt_cleaned_sample = df_3_2[df_3_2['speaker'] == 'Donald Trump'].sample(n=RANDOM_SAMPLE_SIZE, random_state=SEED)
hc_cleaned_sample = df_3_2[df_3_2['speaker'] == 'Hillary Clinton'].sample(n=RANDOM_SAMPLE_SIZE, random_state=SEED)

#### Patches for the Visualisations

In [ ]:
blue_patch = mpatches.Patch(color='blue', alpha=0.4, label='Hillary Clinton')
red_patch = mpatches.Patch(color='red', alpha=0.4, label='Donald Trump')

### 4.1 General Exploration

In [ ]:
df_4_1 = df_3_2

n_bins = 52

plt.figure(figsize=(10,5))

plt.title("Histogram of all Quotes by H.C. and D.T during 2016")

df_4_1_hc = df_4_1[df_4_1['speaker'] == 'Hillary Clinton']
df_4_1_dt = df_4_1[df_4_1['speaker'] == 'Donald Trump']

plt.hist([df_4_1_hc['date'], df_4_1_dt['date']], weights=[df_4_1_hc['numOccurrences'], df_4_1_dt['numOccurrences']], bins=n_bins, color=["blue", "red"], alpha=0.4)

plt.legend(handles=[blue_patch, red_patch])

plt.show()
plt.clf()


**Questions arising**
- Why are there certain periods with barely any quotes?
- Is there something wrong/weird with the data?
- How can we test that everything is good?

## 5. Our Focus Questions

### Q1: Media Bias

Do media outlets portray Trump and Clinton differently? Do media outlets quote the two candidates equally much? Does the bias of the news outlet correlate with the quotes they report?

Let's take two of the biggest outlets with political leaning views, CNN and Breitbart, and compare the distribution of Trump quotes with Clinton quotes, and also see how positive or negative they are.

The NLTK library will also be used for sentiment analysis later on.

In [ ]:
cleaned_sample.groupby(['speaker']).count().sort_values('quotation', ascending=False)['quotation']

**TODO:** Remove the wrong speakers...

#### Using NLTK’s Pre-Trained Sentiment Analyzer

We use NLTK VADER (Valence Aware Dictionary and sEntiment Reasoner) sentiment analysis tools

In [ ]:
sia = SentimentIntensityAnalyzer()

trump_polarity_scores = dt_cleaned_sample['quotation'].apply(sia.polarity_scores)
dt_sample['polarityScore'] = [score.get('compound') for score in trump_polarity_scores]

clinton_polarity_scores = hc_cleaned_sample['quotation'].apply(sia.polarity_scores)
hc_sample['polarityScore'] = [score.get('compound') for score in clinton_polarity_scores]

In [ ]:
trump_df = dt_cleaned_sample
clinton_df = hc_cleaned_sample

cnn_trump, cnn_clinton = [],[]

for i in range(len(trump_df)):
    for k in range(len(trump_df['urls'].iloc[i])):
        if trump_df['urls'].iloc[i][k].find('cnn') != -1:
            cnn_trump.append(i)

trump_cnn = trump_df.iloc[cnn_trump]

for i in range(len(clinton_df)):
    for k in range(len(clinton_df['urls'].iloc[i])):
        if clinton_df['urls'].iloc[i][k].find('cnn') != -1:
            cnn_clinton.append(i)

clinton_cnn = clinton_df.iloc[cnn_clinton]
        
trump_cnn_PS = trump_cnn['quotation'].apply(sia.polarity_scores)
trump_cnn['polarityScore'] = [score.get('compound') for score in trump_cnn_PS]
clinton_cnn_PS = clinton_cnn['quotation'].apply(sia.polarity_scores)
clinton_cnn['polarityScore'] = [score.get('compound') for score in clinton_cnn_PS]

trump_cnn_pos = trump_cnn[trump_cnn['polarityScore'] >= 0]
trump_cnn_neg = trump_cnn[trump_cnn['polarityScore'] < 0]
clinton_cnn_pos = clinton_cnn[clinton_cnn['polarityScore'] >= 0]
clinton_cnn_neg = clinton_cnn[clinton_cnn['polarityScore'] < 0]

fig, axs = plt.subplots(1, 2, figsize=(15, 5))

axs[0].scatter(x=trump_cnn_pos['date'], y=trump_cnn_pos['polarityScore'], color='g', alpha=0.3, label="positive")
axs[0].scatter(x=trump_cnn_neg['date'], y=trump_cnn_neg['polarityScore'], color='r', alpha=0.3, label="negative")
axs[1].scatter(x=clinton_cnn_pos['date'], y=clinton_cnn_pos['polarityScore'], color='g', alpha=0.3, label="positive")
axs[1].scatter(x=clinton_cnn_neg['date'], y=clinton_cnn_neg['polarityScore'], color='r', alpha=0.3, label="positive")

fig.suptitle("CNN quotes compound polarity scores", fontsize=14)

axs[0].set_ylabel('Frequency')
axs[0].title.set_text('Donald Trump')

axs[1].title.set_text('Hillary Clinton')

axs[0].set_xlabel('Date')
axs[1].set_xlabel('Date')

axs[0].set_ylabel('Polarity score')
axs[1].set_ylabel('Polarity score')

plt.show()
plt.clf()

In [ ]:
breitbart_trump, breitbart_clinton = [],[]

for i in range(len(trump_df)):
    for k in range(len(trump_df['urls'].iloc[i])):
        if trump_df['urls'].iloc[i][k].find('breitbart') != -1:
            breitbart_trump.append(i)

trump_breitbart = trump_df.iloc[breitbart_trump]

for i in range(len(clinton_df)):
    for k in range(len(clinton_df['urls'].iloc[i])):
        if clinton_df['urls'].iloc[i][k].find('breitbart') != -1:
            breitbart_clinton.append(i)

clinton_breitbart = clinton_df.iloc[breitbart_clinton]
        
trump_breitbart_PS = trump_breitbart['quotation'].apply(sia.polarity_scores)
trump_breitbart['polarityScore'] = [score.get('compound') for score in trump_breitbart_PS]
clinton_breitbart_PS = clinton_breitbart['quotation'].apply(sia.polarity_scores)
clinton_breitbart['polarityScore'] = [score.get('compound') for score in clinton_breitbart_PS]

trump_breitbart_pos = trump_breitbart[trump_breitbart['polarityScore'] >= 0]
trump_breitbart_neg = trump_breitbart[trump_breitbart['polarityScore'] < 0]
clinton_breitbart_pos = clinton_breitbart[clinton_breitbart['polarityScore'] >= 0]
clinton_breitbart_neg = clinton_breitbart[clinton_breitbart['polarityScore'] < 0]

fig, axs = plt.subplots(1, 2, figsize=(15, 5))

axs[0].scatter(x=trump_breitbart_pos['date'], y=trump_breitbart_pos['polarityScore'], color='g', alpha=0.3, label="positive")
axs[0].scatter(x=trump_breitbart_neg['date'], y=trump_breitbart_neg['polarityScore'], color='r', alpha=0.3, label="negative")
axs[1].scatter(x=clinton_breitbart_pos['date'], y=clinton_breitbart_pos['polarityScore'], color='g', alpha=0.3, label="positive")
axs[1].scatter(x=clinton_breitbart_neg['date'], y=clinton_breitbart_neg['polarityScore'], color='r', alpha=0.3, label="positive")

fig.suptitle("Breitbart quotes compound polarity scores", fontsize=14)

axs[0].set_ylabel('Frequency')
axs[0].title.set_text('Donald Trump')

axs[1].title.set_text('Hillary Clinton')

axs[0].set_xlabel('Date')
axs[1].set_xlabel('Date')

axs[0].set_ylabel('Polarity score')
axs[1].set_ylabel('Polarity score')

plt.show()
plt.clf()

This might be a very small sample size for the EDA, but we can already see substantial bias both in terms of the number of quotes by a candidate in each media outlet as well as the ratio of positive/negative quotes chosen by the CNN and Breitbart to represent the two Presidential candidates. This most likely holds true for other popular media outlets, especially those with politically inclined views.

### Q2: Political Issues

We want to track the different political topics that the candidates focused on according to the content of their quotes. From this we aim to learn the importance of the topics in a absolut relative context but also in regards to when a certain topic might have been very present and then. disappeared for some time. We furthermore want to see if there might be a candidate which sparked a topic or at least started talking/being quoted about it first.

Bewlow we show an example of a topic in regards to Obamacare, which we all remember to be a important topic during the debate.

In [ ]:
OBAMACARE_REGEX = '(obama care)|obamacare' # Not perfect, proof of concept...

df_obamacare = df_3_2[df_3_2['quotation'].str.contains(pat = OBAMACARE_REGEX, regex = True, flags=re.IGNORECASE)].sort_values('date')
df_obamacare['date'] = df_obamacare['date'].apply(lambda date : date.date())

df_obamacare_hc = df_obamacare[df_obamacare['speaker'] == 'Hillary Clinton']
df_obamacare_dt = df_obamacare[df_obamacare['speaker'] == 'Donald Trump']

In [ ]:
BINS = 52
ticks = [f"2016-0{i * 3}-01" for i in range(1,4)] + ["2016-12-01"]

plt.figure(figsize=(10,5))
plt.title("Obamacare mentions in 2016", fontsize=14)

plt.hist([df_obamacare_hc['date'], df_obamacare_dt['date']], weights=[df_obamacare_hc['numOccurrences'], df_obamacare_dt['numOccurrences']], bins=BINS, color=["b", "r"], alpha=0.4)

plt.xticks(ticks)
plt.legend(handles=[blue_patch, red_patch])

plt.show()
plt.clf()

In [ ]:
def quote_topics_histogram(df_hc, df_dt, bins=52, topic_name=""):
    
    fig, axs = plt.subplots(1, 2, figsize=(10, 5))

    kwargs = {
        "alpha" : 0.4,
        "bins": bins,
    }

    axs[0].hist(df_hc['date'], weights=df_hc['numOccurrences'], color="b", **kwargs)
    axs[1].hist(df_dt['date'], weights=df_dt['numOccurrences'], color="r", **kwargs)

    fig.suptitle(f"Mentions of the Topic '{topic_name}' in 2016 by candidate", fontsize=16)

    axs[0].set_ylabel('Frequency')

    axs[0].title.set_text('Hillary Clinton')
    axs[0].set_xlabel('Date')

    axs[1].title.set_text('Donald Trump')
    axs[1].set_xlabel('Date')
    
    ticks = [f"2016-0{i * 3}-01" for i in range(1,4)] + ["2016-12-01"]

    axs[0].set_xticks(ticks)
    axs[1].set_xticks(ticks)
    
    plt.show()
    plt.clf()

In [ ]:
# Example:
quote_topics_histogram(df_obamacare_hc, df_obamacare_dt, topic_name="Obamacare")

#### Important Interrlude

Hillary Clinton talked less about Obamacare but instead more about Healthcare... To be explored further!

### Visualisation Idea

We have identified several topics which show a clear periodic difference in occurance and intensity. We aim to visualise this data as illustrated in the sketch below for the final project. We hope to show how several topics come and go throughout the campaign and are focused on with a different amount of attention/importance depending on the candidate. Ideally we would extend this visualisation further allowing the viewer to see the importance the citizen base gave the topics at each time or the extent to which media reported on it. It would be amazing to see if some topics had a trendsetter who started talking about it earier adn fiercly...

We believe that our first EDA supports our thesis that the data will allow us to visualise it in this way, that there is peridoicity and hoepefully many interesting insights.

![Visualisation Sketch](focus-topics-sketch.jpg)

## Q3: Language

### a) Intellectuality of Language

Idea 1: Count the syllables in each candidate's consolidated quotes and compare distributions

In [ ]:
def syllables_hist(dt_text: str, hc_text: str):

    dt_syl = list(map(syllables.estimate, dt_text.split()))
    hc_syl = list(map(syllables.estimate, hc_text.split()))

    kwargs = {
        "x"    : [hc_syl, dt_syl],
        "label" : ["Hillary Clinton", "Donald Trump"],
        "color" : ["b", "r"],
        "alpha" : 0.4,
        "bins" : range(1, 6),
        "density" : True,
        "align": "left",
    }

    plt.hist(**kwargs)
    plt.legend()
    plt.title("Histogram of number of syllables", fontsize=16)
    plt.xlabel("Number of syllables in a Word")
    plt.ylabel("Probability of Occuring")

    # TODO generalize this
    plt.xticks(range(1, 5), range(1, 5))

    plt.show()

In [ ]:
hc_text = ' '.join(hc_cleaned_sample["quotation"])
dt_text = ' '.join(dt_cleaned_sample["quotation"])

In [ ]:
syllables_hist(dt_text, hc_text)

Idea 2: Look at the CEFR language level. We can measure the language level for all words in each candidate's consolidated quotes and compare the distributions.

But first we need to get data on words. We have the HTML for a web page containing these data, and we can put them in the form of a CSV file.

In [ ]:
def bs_to_csv(soup, csv_writer):
    table = soup.find("tbody")
    tr_rows = table.find_all("tr")
    csv_writer.writerow(["word", "guideword", "level", "part of speech", "topic"])
    csv_writer.writerows(
        [[t.text for t in r.find_all("td")][:-1] for r in tr_rows]
    )

In [ ]:
# Scrape the HTML file and write in CSV format to file.
with open(CEFR_HTML_IN) as fp, open(CEFR_CSV_OUT, 'w') as csv_out:
    soup = BeautifulSoup(fp)
    csv_writer = csv.writer(csv_out, delimiter=',')
    bs_to_csv(soup, csv_writer)

In [ ]:
df_cefr = pd.read_csv(CEFR_CSV_OUT)

display(df_cefr)

The data includes idioms as well as nouns; this is intelligible for a human, but it might be a bit of trouble to make it work here, so we remove them.
We'll only need the `word` and `level` columns, so we remove the others as well.

In [ ]:
df_cefr = df_cefr[[len(w.split()) == 1 for w in df_cefr["word"]]]\
                 .filter(items=["word", "level"])

Now we set everything to lowercase. Also it seems that some words have punctuation surrounding them, which is not desirable.

In [ ]:
df_cefr["word"] = df_cefr["word"].transform(
    lambda w: w.lower()\
               .translate(str.maketrans('', '', string.punctuation)))

display(df_cefr.head())

Also, the same word can appear multiple times (as there might be different phrases in which the word means something slightly different) so we aggregate the levels using the median level.

In [ ]:
cefr_level_map = {
    "A1": 1,
    "A2": 2,
    "B1": 3,
    "B2": 4,
    "C1": 5,
    "C2": 6,
}

In [ ]:
df_cefr_copy = df_cefr.copy(deep=True)

In [ ]:
# Use the map to transform the level column

df_cefr.level = df_cefr_copy["level"].map(cefr_level_map)

In [ ]:
# Aggregate with the median

df_cefr = df_cefr.groupby("word").agg("median").reset_index()

display(df_cefr.head())

Finally we change the index to the word itself to facilitate searching.

In [ ]:
df_cefr.index = df_cefr.word
df_cefr = df_cefr.filter(items=["level"])

Now we can finally apply this to our dataset.

In [ ]:
cefr_data = pd.read_csv(CEFR_CLEAN_CSV_IN, index_col="word")
cefr_data.loc["he"].loc["level"]

In [ ]:
def cefr_level_hist(dt_text: str, hc_text: str):

    cefr_data = pd.read_csv(CEFR_CLEAN_CSV_IN, index_col="word")
    cefr_data.loc["he"].loc["level"]
    
    level = lambda w: cefr_data.loc[w].loc["level"] if w in cefr_data.index else 0

    dt_level = list(map(level, dt_text.split()))
    hc_level = list(map(level, hc_text.split()))

    kwargs = {
        "x"    : [hc_level, dt_level],
        "label" : ["Hillary Clinton", "Donald Trump"],
        "color" : ["b", "r"],
        "alpha" : 0.4,
        "bins" : range(0, 7),
        "density" : True,
        "align": "left",
    }
    plt.hist(**kwargs)
    plt.legend()
    plt.title("Histogram of English level of words", fontsize=16)
    plt.xlabel("English level")
    plt.ylabel("Probability")

    plt.xticks(range(0, 7), ["NA", "A1", "A2", "B1", "B2", "C1", "C2"])

    plt.show()

In [ ]:
cefr_level_hist(dt_text.lower(), hc_text.lower())

There seems to be little difference in terms of language levels, but note how many words were not accounted for; this is due in part to limitations of our CEFR dataset, and also to the fact that some words appear in text in different forms (e.g. a verb conjugated to the past tense). This particular issue will be remedied by "stemming" the words to reduce them to their "base" form.

### Ideas

- Who is are Trump and Clinton most confused with? Using the other speaker attributions in the quotas list.

### b) Sentiment Analysis

In [ ]:
cleaned_sample.groupby(['speaker']).count().sort_values('quotation', ascending=False)['quotation']

##### Using NLTK’s Pre-Trained Sentiment Analyzer
We use NLTK VADER (Valence Aware Dictionary and sEntiment Reasoner) sentiment analysis tools

In [ ]:
sia = SentimentIntensityAnalyzer()

In [ ]:
trump_polarity_scores = dt_cleaned_sample['quotation'].apply(sia.polarity_scores)
dt_cleaned_sample['polarityScore'] = [score.get('compound') for score in trump_polarity_scores]

clinton_polarity_scores = hc_cleaned_sample['quotation'].apply(sia.polarity_scores)
hc_cleaned_sample['polarityScore'] = [score.get('compound') for score in clinton_polarity_scores]

In [ ]:
def print_topk_quotes(df, sentiment, k):
    if (sentiment == "positive"):
        [print("\n[+]", quote) for quote in df.sort_values('polarityScore', ascending=False)['quotation'].head(k).tolist()]
    else:
        [print("\n[-]", quote) for quote in df.sort_values('polarityScore', ascending=True)['quotation'].head(k).tolist()]

#### Donald Trump's 5 most positive and negative quotes

In [ ]:
print("Trump's 5 most positive quotes:")
# trump_df.sort_values('polarityScore', ascending=False)[['quotation', 'polarityScore']].head(5)
[print("\n[+]", quote) for quote in dt_cleaned_sample.sort_values('polarityScore', ascending=False)['quotation'].head(5).tolist()]

print("\n---\n")

print("Trump's 5 most negative quotes:")
# trump_df.sort_values('polarityScore', ascending=True)[['quotation', 'polarityScore']].head(5).tolist()
[print("\n[-]", quote) for quote in dt_cleaned_sample.sort_values('polarityScore', ascending=True)['quotation'].head(5).tolist()]

#### Hillary Clinton's 5 most positive and negative quotes

In [ ]:
print("Hillary Clinton's 5 most positive quotes:")
# clinton_df.sort_values('polarityScore', ascending=False)[['quotation', 'polarityScore']].head(5)
[print("\n[+]", quote) for quote in hc_cleaned_sample.sort_values('polarityScore', ascending=False)['quotation'].head(5).tolist()]

print("\n---\n")

print("Hillary Clinton's 5 most negative quotes:")
# clinton_df.sort_values('polarityScore', ascending=True)[['quotation', 'polarityScore']].head(5).tolist()
[print("\n[-]", quote) for quote in hc_cleaned_sample.sort_values('polarityScore', ascending=True)['quotation'].head(5).tolist()]


In [ ]:
trump_df_pos = dt_sample[dt_sample['polarityScore'] >= 0]
trump_df_neg = dt_sample[dt_sample['polarityScore'] < 0]

plt.scatter(x=trump_df_pos['date'], y=trump_df_pos['polarityScore'], color='g', alpha=0.3, label="positive")
plt.scatter(x=trump_df_neg['date'], y=trump_df_neg['polarityScore'], color='r', alpha=0.3, label="negative")

plt.title("Trump's quotes compound polarity scores by date in 2016")
plt.xlabel('Date')
plt.ylabel('Polarity score')
plt.legend()

plt.show()

In [ ]:
clinton_df_pos = hc_sample[hc_sample['polarityScore'] >= 0]
clinton_df_neg = hc_sample[hc_sample['polarityScore'] < 0]

plt.scatter(x=clinton_df_pos['date'], y=clinton_df_pos['polarityScore'], color='g', alpha=0.3, label="positive")
plt.scatter(x=clinton_df_neg['date'], y=clinton_df_neg['polarityScore'], color='r', alpha=0.3, label="negative")

plt.title("Clinton's quotes compound polarity scores by date in 2016")
plt.xlabel('Date')
plt.ylabel('Polarity score')
plt.legend()

plt.show()

#### Distribution of Hillary Clinton and Donald Trumps compound sentiment

In [ ]:
# plot with matplotlib
fig, ax = plt.subplots(1,2, figsize=(12,6),  sharey=True, sharex=True)

bins = 40 
kwargs = {
    "alpha" : 0.4,
    "bins": bins,
}
    
# clinton compound polarity distribution
ax[0].hist(hc_sample['polarityScore'], color="b", **kwargs)
ax[0].set_title("Hilary Clinton")
ax[0].set_xlabel('Compound sentiment')
ax[0].set_ylabel('Number of quotes')

# trump compound polarity distribution
ax[1].hist(dt_sample['polarityScore'], color="r", **kwargs)
ax[1].set_title("Donald Trump")
ax[1].set_xlabel('Compound sentiment')
# ax[1].set_ylabel('Number of quotes')
    
fig.suptitle("Hilary Clinton VS Donald Trump quotes compound polarity distribution in 2016", fontweight="bold")
fig.tight_layout()
plt.show()

In [ ]:
# plot with plotly
hc_dt_sample = pd.DataFrame(dict(
    candidate = np.concatenate((["Hillary Clinton"]*len(hc_sample), 
                             ["Donald Trump"]*len(dt_sample))), 
    polarityScore   = np.concatenate((hc_sample['polarityScore'],
                                      dt_sample['polarityScore']))
))

fig = px.histogram(hc_dt_sample, x="polarityScore", color="candidate", 
                   barmode="overlay", nbins=40, marginal="box",
                   width=750, height=500,
                   log_y=True
)

# customize font and legend orientation & position
fig.update_layout( 
    font_family="Rockwell",
    legend=dict(
        title=None, orientation="h", y=1, yanchor="bottom", x=0.5, xanchor="center"
    ),
    title_text='Hilary Clinton VS Donald Trump quotes compound polarity distribution in 2016', # title of plot
    xaxis_title_text='Sentiment (polarity compound score)',
    yaxis_title_text='Number of quotes (log scale)', 
    # bargap=0.2, # gap between bars of adjacent location coordinates
    bargroupgap=0.05 # gap between bars of the same location coordinates
)
fig.show()

In [ ]:
# create a list of our conditions
sentiment_conditions = [
    (hc_dt_sample['polarityScore'] <= -0.05),
    (np.abs(hc_dt_sample['polarityScore']) < 0.05),
    (hc_dt_sample['polarityScore'] >= 0.05)
]

# create a list of the values we want to assign for each condition
values = ['negative', 'neutral', 'positive']

# create a new column and use np.select to assign values to it using our lists as arguments
hc_dt_sample['sentiment'] = np.select(sentiment_conditions, values)

# display updated DataFrame
# display(hc_dt_sample.head())

# aggregate our dataframe by sentiment and candidate
df_sentiment_agg = hc_dt_sample.groupby(['sentiment', 'candidate']).size().reset_index(name='counts')

# sort to have Clinton values first
df_sentiment_agg.sort_values('candidate', ascending=False, inplace=True)

fig = px.bar(df_sentiment_agg, x="sentiment", y="counts",
             color='candidate', barmode='group',
             color_discrete_map={
                'Hillary Clinton': px.colors.qualitative.Plotly[0],
                'Donald Trump': px.colors.qualitative.Plotly[1],
             },
             height=400,
            text="counts"
            )

# customize font and legend orientation & position
fig.update_layout( 
    font_family="Rockwell",
    legend=dict(
        title=None, orientation="h", y=1, yanchor="bottom", x=0.5, xanchor="center"
    ),
    title_text='Overall Sentiment of Clinton vs Trump quotes in 2016', title_x=0.5,
    xaxis_title_text='Sentiment',
    yaxis_title_text='Number of quotes', 
    # bargap=0.2, # gap between bars of adjacent location coordinates
    # bargroupgap=0.05 # gap between bars of the same location coordinates
)
fig.show()

print("\n(Negative score < -0.05, Neutral score: -0.05 < score < 0.05, Positive score > 0.05")

In [ ]:
# calculate totals
dt_tot_quotes_count = len(dt_sample)
dt_pos_quotes_count = sum(dt_sample['polarityScore']>=0.05)
dt_neg_quotes_count = sum(dt_sample['polarityScore']<=-0.05)
dt_neu_quotes_count = sum(np.abs(dt_sample['polarityScore'])<0.05)

hc_tot_quotes_count = len(hc_sample)
hc_pos_quotes_count = sum(hc_sample['polarityScore']>=0.05)
hc_neg_quotes_count = sum(hc_sample['polarityScore']<=-0.05)
hc_neu_quotes_count = sum(np.abs(hc_sample['polarityScore'])<0.05)

# dt_pos_quotes_count = df_sentiment_agg['counts'].loc[(df_sentiment_agg['candidate'] == 'Donald Trump') & (df_sentiment_agg['sentiment'] == 'positive')]
# dt_neg_quotes_count = df_sentiment_agg['counts'].loc[(df_sentiment_agg['candidate'] == 'Donald Trump') & (df_sentiment_agg['sentiment'] == 'negative')]
# dt_neu_quotes_count = df_sentiment_agg['counts'].loc[(df_sentiment_agg['candidate'] == 'Donald Trump') & (df_sentiment_agg['sentiment'] == 'neutral')]
# dt_tot_quotes_count = dt_pos_quotes_count + dt_neg_quotes_count + dt_neu_quotes_count

# hc_pos_quotes_count = df_sentiment_agg.loc[(df_sentiment_agg['candidate'] == 'Hillary Clinton') & (df_sentiment_agg['sentiment'] == 'positive')]
# hc_neg_quotes_count = df_sentiment_agg.loc[(df_sentiment_agg['candidate'] == 'Hillary Clinton') & (df_sentiment_agg['sentiment'] == 'negative')]
# hc_neu_quotes_count = df_sentiment_agg.loc[(df_sentiment_agg['candidate'] == 'Hillary Clinton') & (df_sentiment_agg['sentiment'] == 'neutral')]
# hc_tot_quotes_count = hc_pos_quotes_count + hc_neg_quotes_count + hc_neu_quotes_count


# summary
print()
print('                                     Hilary Clinton     |      Donald Trump      ')
print('                                ------------------------|------------------------')
print('Total number quotes in sample:        {:>4} ({:.1%})     |      {:>4} ({:.1%})'.format(hc_tot_quotes_count, hc_tot_quotes_count/hc_tot_quotes_count,
                                                                                               dt_tot_quotes_count, dt_tot_quotes_count/dt_tot_quotes_count))
print('Number of negative quotes:            {:>4} ({:.1%})      |      {:>4} ({:.1%})'.format(hc_neg_quotes_count, hc_neg_quotes_count/hc_tot_quotes_count,
                                                                                                dt_neg_quotes_count, dt_neg_quotes_count/dt_tot_quotes_count ))
print('Number of neutral quotes:             {:>4} ({:.1%})      |      {:>4} ({:.1%})'.format(hc_neu_quotes_count, hc_neu_quotes_count/hc_tot_quotes_count,
                                                                                                dt_neu_quotes_count, dt_neu_quotes_count/dt_tot_quotes_count))
print('Number of positive quotes:            {:>4} ({:.1%})      |      {:>4} ({:.1%})'.format(hc_pos_quotes_count, hc_pos_quotes_count/hc_tot_quotes_count,
                                                                                                dt_pos_quotes_count, dt_pos_quotes_count/dt_tot_quotes_count ))

print("\n(Positive score > 0.05, Negative score < -0.05, Neutral score: -0.05 < score < 0.05)")

### Sentiment analysis by topic

In [ ]:
def filter_n_concatenate_dfs(hc_sample, dt_sample, topic_keywords):
    hc_filtered = hc_sample[hc_sample['quotation'].str.contains(topic_keywords, case=False)]
    dt_filtered = dt_sample[dt_sample['quotation'].str.contains(topic_keywords, case=False)]
    
    hc_dt_filtered = pd.DataFrame(dict(
        candidate = np.concatenate((["Hillary Clinton"]*len(hc_filtered), 
                                 ["Donald Trump"]*len(dt_filtered))), 
        polarityScore   = np.concatenate((hc_filtered['polarityScore'],
                                          dt_filtered['polarityScore'])),
        quotation   = np.concatenate((hc_filtered['quotation'],
                                          dt_filtered['quotation']))
    ))
    return hc_dt_filtered    

In [ ]:
def sentimentHistogram(hc_sample, dt_sample, topic_keywords): 
    hc_dt_filtered = filter_n_concatenate_dfs(hc_sample, dt_sample, topic_keywords)
    
    # plot with plotly
    fig = px.histogram(hc_dt_filtered, x="polarityScore", color="candidate", 
                       barmode="overlay", nbins=40, marginal="box",
                       width=750, height=500,
                       # log_y=True
    )

    # customize font and legend orientation & position
    fig.update_layout( 
        font_family="Rockwell",
        legend=dict(
            title=None, orientation="h", y=1, yanchor="bottom", x=0.5, xanchor="center"
        ),
        title_text='Sentiment anlysis on topic: '+topic_keywords, title_x=0.5,
        xaxis_title_text='Sentiment (polarity compound score)',
        yaxis_title_text='Number of quotes (log scale)', 
        # bargap=0.2, # gap between bars of adjacent location coordinates
        bargroupgap=0.05, # gap between bars of the same location coordinates
    )
    fig.update_xaxes(range=[-1, 1])
    fig.show()


In [ ]:
def candidateBarChart(hc_sample, dt_sample, topic_keywords): 
    hc_dt_filtered = filter_n_concatenate_dfs(hc_sample, dt_sample, topic_keywords)

    quote_counts = hc_dt_filtered.groupby(['candidate']).size().reset_index(name='counts')
    hc_quote_counts = quote_counts[quote_counts['candidate'] == 'Hillary Clinton'].counts
    dt_quote_counts = quote_counts[quote_counts['candidate'] == 'Donald Trump'].counts

    fig = px.bar(quote_counts, x="candidate", y="counts",
                 color='candidate',
                 color_discrete_map={
                    'Hillary Clinton': px.colors.qualitative.Plotly[0],
                    'Donald Trump': px.colors.qualitative.Plotly[1],
                 },
                 height=400,
                text="counts"
                )
    # customize font and legend orientation & position
    fig.update_layout( 
        font_family="Rockwell",
        legend=dict(
            title=None, orientation="h", y=1, yanchor="bottom", x=0.5, xanchor="center"
        ),
        title_text='Total number of quotes about :'+topic_keywords, title_x=0.5,
        xaxis_title_text='Candidate',
        yaxis_title_text='Number of quotes', 
        # bargap=0.2, # gap between bars of adjacent location coordinates
        # bargroupgap=0.05 # gap between bars of the same location coordinates
    )
    fig.show()

In [ ]:
def sentimentBarChart(hc_sample, dt_sample, topic_keywords): 
    hc_dt_filtered = filter_n_concatenate_dfs(hc_sample, dt_sample, topic_keywords)

    # create a list of our conditions
    sentiment_conditions = [
        (hc_dt_filtered['polarityScore'] <= -0.05),
        (np.abs(hc_dt_filtered['polarityScore']) < 0.05),
        (hc_dt_filtered['polarityScore'] >= 0.05)
    ]

    # create a list of the values we want to assign for each condition
    values = ['negative', 'neutral', 'positive']

    # create a new column and use np.select to assign values to it using our lists as arguments
    hc_dt_filtered['sentiment'] = np.select(sentiment_conditions, values)

    # aggregate our dataframe by sentiment and candidate
    df_sentiment_agg = hc_dt_filtered.groupby(['sentiment', 'candidate']).size().reset_index(name='counts')

    # NORMALIZATION

    # compute total number of quotes per candidate
    # quote_counts = hc_dt_filtered.groupby(['candidate']).size()
    # hc_quote_counts = quote_counts['Hillary Clinton']
    # dt_quote_counts = quote_counts['Donald Trump']

    quote_counts = hc_dt_filtered.groupby(['candidate']).size().reset_index(name='counts')
    hc_quote_counts = quote_counts[quote_counts['candidate'] == 'Hillary Clinton'].counts
    dt_quote_counts = quote_counts[quote_counts['candidate'] == 'Donald Trump'].counts

    hc_quote_counts = hc_quote_counts.values[0]
    dt_quote_counts = dt_quote_counts.values[0]

    # divide by total number of quotes per candidate to get percentage
    df_sentiment_agg['counts_pct'] = 0
    df_sentiment_agg['counts_pct'] = np.where(df_sentiment_agg['candidate'] == 'Donald Trump', df_sentiment_agg['counts']/dt_quote_counts, df_sentiment_agg['counts_pct'])
    df_sentiment_agg['counts_pct'] = np.where(df_sentiment_agg['candidate'] == 'Hillary Clinton', df_sentiment_agg['counts']/hc_quote_counts, df_sentiment_agg['counts_pct'])

    # sort to have Clinton values first
    df_sentiment_agg.sort_values('candidate', ascending=False, inplace=True)

    fig = px.bar(df_sentiment_agg, x="sentiment", y="counts_pct",
                 color='candidate', barmode='group',
                 color_discrete_map={
                    'Hillary Clinton': px.colors.qualitative.Plotly[0],
                    'Donald Trump': px.colors.qualitative.Plotly[1],
                 },
                 height=400,
                text="counts_pct"
                )

    # customize font and legend orientation & position
    fig.update_layout( 
        font_family="Rockwell",
        legend=dict(
            title=None, orientation="h", y=1, yanchor="bottom", x=0.5, xanchor="center"
        ),
        title_text='Overall Sentiment of Clinton vs Trump about: '+topic_keywords, title_x=0.5,
        xaxis_title_text='Sentiment',
        yaxis_title_text='% of quotes per candidate', 
        # bargap=0.2, # gap between bars of adjacent location coordinates
        # bargroupgap=0.05 # gap between bars of the same location coordinates
    )
    fig.update_traces(texttemplate='%{text:.0%}', textposition='inside')
    
    return fig

    fig.show()

    print("\n(Negative score < -0.05, Neutral score: -0.05 < score < 0.05, Positive score > 0.05")


In [ ]:
topic_keywords = 'economy'
# sentimentHistogram(hc_sample, dt_sample, topic_keywords)
candidateBarChart(hc_sample, dt_sample, topic_keywords)
sentimentBarChart(hc_sample, dt_sample, topic_keywords)

In [ ]:
topic_keywords = 'immigration|immigrant|immigrants|border'
# sentimentHistogram(hc_sample, dt_sample, topic_keywords)
sentimentBarChart(hc_sample, dt_sample, topic_keywords)

In [ ]:
topic_keywords = 'obamacare'
# sentimentHistogram(hc_sample, dt_sample, topic_keywords)
sentimentBarChart(hc_sample, dt_sample, topic_keywords)

In [ ]:
topic_keywords = 'abortion'
# sentimentHistogram(hc_sample, dt_sample, topic_keywords)
sentimentBarChart(hc_sample, dt_sample, topic_keywords)
hc_dt_filtered = filter_n_concatenate_dfs(hc_sample, dt_sample, topic_keywords)

print_topk_quotes(hc_dt_filtered, "positive", 3)

### Sentiment analysis on the target of quotes. 

For instance, Trump calling Clinton "Crooked Hillary" would be a negative statement about Presidential candidate Clinton. This will be done for a number of politicians, including: Hillary Clinton, Nancy Pelosi, Barack Obama, Bernie Sanders, Elizabeth Warren, Ted Cruz, Mike Pence, and Mitch McConnell.

In [ ]:
trump_df = sample.loc[sample['speaker'] == 'Donald Trump']
clinton_df = sample.loc[sample['speaker'] == 'Hillary Clinton']

trump_on_clinton = trump_df[trump_df['quotation'].str.contains('clinton|hillary', case=False)]
trump_on_obama = trump_df[trump_df['quotation'].str.contains('obama|barack', case=False)]
trump_on_sanders = trump_df[trump_df['quotation'].str.contains('bernie|sanders', case=False)]

clinton_on_trump = clinton_df[clinton_df['quotation'].str.contains('trump|donald', case=False)]

In [ ]:
trump_on_clinton_PS = trump_on_clinton['quotation'].apply(sia.polarity_scores)
trump_on_clinton['polarityScore'] = [score.get('compound') for score in trump_on_clinton_PS]

trump_on_obama_PS = trump_on_obama['quotation'].apply(sia.polarity_scores)
trump_on_obama['polarityScore'] = [score.get('compound') for score in trump_on_obama_PS]

In [ ]:
trump_on_clinton_pos = trump_on_clinton[trump_on_clinton['polarityScore'] >= 0]
trump_on_clinton_neg = trump_on_clinton[trump_on_clinton['polarityScore'] < 0]

plt.scatter(x=trump_on_clinton_pos['date'], y=trump_on_clinton_pos['polarityScore'], color='g', alpha=0.3, label="positive")
plt.scatter(x=trump_on_clinton_neg['date'], y=trump_on_clinton_neg['polarityScore'], color='r', alpha=0.3, label="negative")

plt.title("Trump's Hillary-targeted quotes compound polarity scores by date in 2016")
plt.xlabel('Date')
plt.ylabel('Polarity score')
plt.legend()

plt.show()

In [ ]:
trump_on_obama_pos = trump_on_obama[trump_on_obama['polarityScore'] >= 0]
trump_on_obama_neg = trump_on_obama[trump_on_obama['polarityScore'] < 0]

plt.scatter(x=trump_on_obama_pos['date'], y=trump_on_obama_pos['polarityScore'], color='g', alpha=0.3, label="positive")
plt.scatter(x=trump_on_obama_neg['date'], y=trump_on_obama_neg['polarityScore'], color='r', alpha=0.3, label="negative")

plt.title("Trump's Obama-targeted quotes compound polarity scores by date in 2016")
plt.xlabel('Date')
plt.ylabel('Polarity score')
plt.legend()

plt.show()

Perhaps a next step would be to compare this with a baseline ratio of positive/negative words. For instance, maybe on average people say 70% positive things and 30% negative. Knowing this could help compare results with Trump and Clinton.

### Using BERT for sentiment analysis using quote vector embedding

In [ ]:
# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
trump_df['quotation'].iloc[500]

In [ ]:
# Add the special tokens.
trump_df['marked'] = "[CLS] " + trump_df['quotation'] + " [SEP]"

trump_df['tokenized'] = trump_df['marked']
trump_df['indexed'] = trump_df['marked']
trump_df['seg_ids'] = trump_df['marked']
trump_df['tokens_tensor'] = trump_df['marked']
trump_df['seg_tensors'] = trump_df['marked']

# Split the sentence into tokens.
for i in range(len(trump_df)):
    trump_df['tokenized'].iloc[i] = tokenizer.tokenize(trump_df['marked'].iloc[i])
    # Map the token strings to their vocabulary indeces.
    trump_df['indexed'].iloc[i] = tokenizer.convert_tokens_to_ids(trump_df['tokenized'].iloc[i])
    trump_df['seg_ids'].iloc[i] = [1] * len(trump_df['tokenized'].iloc[i])
    # Convert inputs to PyTorch tensors
    trump_df['tokens_tensor'].iloc[i] = torch.tensor([trump_df['indexed'].iloc[i]])
    trump_df['seg_tensors'].iloc[i] = torch.tensor([trump_df['seg_ids'].iloc[i]])

#### BERT Next Steps

The next step is to evaluate BERT on our quotations, and fetch the hidden states of the network. We would then create word and sentence vectors from our hidden states. This creates 768-dimensional token embedding vectors, on which we can then apply principal component analysis to plot the results using 2 axes.